In [3]:
from scipy import signal
from scipy.fft import fftshift
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import soundfile as sf
import os
import re
from IPython.display import Audio
#
# Given a signal x break it up into segments where we think the speaker 
# is speaking.
#
# First, take absolute value and normalize by dividing max value
#
# Repeat the following  
#
#     1) start interval at first value that is above initial_threshold
#     2) end interval at the first value starts at a window of duration wdur seconds
#        has its max below the kill threshold
#
# Remove all intervals whose duration is less than mindur seconds
#
def find_gaps(x,mindur,wdur, initial_threshold, kill_threshold):
    #
    # compute number of indices for mindur and wsecs
    #
    mindur_indices=int(mindur*44100)
    wdur_indices=int(wdur*44100)
    #
    #
    #
    # print("mindur_indices = "+str(mindur_indices))
    # print("wdur_indices= "+str(wdur_indices))
#     print("xlength = "+str(len(x)))
    
    #
    # normalize the signal
    #
    x=np.abs(x)
    M=np.max(x)
    x=x/M
    xt=x>=initial_threshold
    # print("number of x's above threshold ="+str(xt.sum()))
    #
    # compute indicator of max in windows below threshold
    #
    y=pd.Series(x).rolling(wdur_indices).max()
    y.to_csv("y_4_3.csv")
    #
    # remove the initial na's and find whose maximum is below threshold
    #
    y=np.array(y.iloc[wdur_indices-1:]<kill_threshold)
    #print("sum of y = "+str(np.sum(y)))
    pd.DataFrame({"xt":xt[0:len(y)],"y":y}).to_csv("out_4_3.csv")
    # print(y)
    # print("ylength = "+str(len(y)))
    # #
    # find next interval starting at position pos
    #
    def find_next_interval(x,y,pos,Nx,Ny):
        # print("in find next interval sum of x's "+str(x.sum()))
        # print(x)
        flag=False
        for i in range(pos,Nx):
            #print(x[i])
            if x[i]:
                flag=True
                break
        if not flag:
            return(None)
        start=i
        for i in range(start+1,Ny):
            if y[i]:
                end=i
                return((start,end))
        return((start,Nx))
    pos=0
    Nx=len(xt)
    Ny=len(y)
    L=[]
    while True:
        res=find_next_interval(xt,y,pos,Nx,Ny)
        if not res:
            return(L)
        pos=res[1]
        #print(res)
        #print(res[0]/44100,res[1]/44100)
        if res[1]-res[0]>=mindur_indices:
            L.append(res)
            #print(res[0]/44100,res[1]/44100)
    return(L)

wavefile_folder="addclip/"
L=os.listdir(wavefile_folder)
L=[x for x in L if re.search("wav",x)]
# print(L)

def numb_to_string(n):
    if n<10:
        return("00"+str(n))
    if n<100:
        return("0"+str(n))

temp=[]
for x in L:
    fileid=wavefile_folder+x
    # print(fileid)
    name=x.replace(".wav","")
    x,samplerate=sf.read(fileid)
    gaps=find_gaps(x,mindur=.5,wdur=.3,initial_threshold=.1,kill_threshold=.01)
    ctr=0
    for g in gaps:
        snippet=x[g[0]:g[1]]
        sf.write("snippets4_3/snippet_"+name+numb_to_string(ctr)+".wav",snippet,samplerate=44100)
        ctr+=1
    temp.append(ctr)
print(temp)

[45, 45]


In [ ]:

from scipy import signal
from scipy.fft import fftshift
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import soundfile as sf
import os
import re
from IPython.display import Audio
rng = np.random.default_rng()

wavefile_folder="snippets2/"
L=os.listdir(wavefile_folder)
L=[x for x in L if re.search("wav",x)]
#print(L)

fileid=wavefile_folder+L[3]
print(fileid)
x,samplerate=sf.read(fileid)
print(samplerate)
print(x.shape)
plt.plot(x)